In [6]:
import pandas as pd
import numpy as np
import easier as ezr
import lifelines as ll
%config Completer.use_jedi = False

In [103]:
class Loader(ezr.pickle_cache_mixin):
    pkc = ezr.pickle_cache_state('active')
    
    @ezr.pickle_cached_container()
    def df_raw(self):
        gsheet = ezr.GSheet('pseudoscorpion_data_sheet', 'treatments')
        df = gsheet.to_dataframe()
        df.columns = ezr.slugify(df.columns)
        return df
    
    @ezr.cached_container
    def df(self):
        df = self.df_raw
        
        df.loc[:, 'start_date'] = df.start_date.astype(np.datetime64)
        df = df.set_index(['id', 'trial', 'start_date'])
        df = df.T.reset_index().rename(columns={'index': 'observation_date'}).sort_index()
        # df = df.sort_index(axis=1)
        df.iloc[:, 0] = (df.iloc[:, 0].str.replace('_', ' ') + ' 2020').astype(np.datetime64)
        df = df.replace(r'^\s*$', np.NaN, regex=True)
        # df = df.set_index[('observation_date', '')]
        df = df.set_index(('observation_date', '', np.NaN))

        df.index.name = 'date'

        lol = []
        for col_ind in range(df.shape[1]):
            ser = df.iloc[:, col_ind].dropna()
            sample_id = df.columns[col_ind][0]
            treatment = df.columns[col_ind][1]
            term_date = ser.index[-1]
            is_alive = ser[-1]
            start_date = df.columns[col_ind][-1]
            lol.append([sample_id, treatment, start_date, term_date, is_alive])

        dfx = pd.DataFrame(lol, columns=['sample_id', 'treatment', 'start_date', 'term_date', 'is_alive'])
        dfx['duration'] = (dfx.term_date - dfx.start_date).dt.days
        dfx['died'] = (1 - dfx.is_alive).astype(int)
        dfx.loc[:, 'sample_id'] = dfx.sample_id.str.strip().str.replace(r'\s+', '', regex=True)
        dfx.loc[:, 'treatment'] = dfx.treatment.str.strip().str.replace(r'\s+', '', regex=True)
        dfx = dfx[['sample_id', 'treatment', 'duration', 'died']]
        return dfx
        

In [105]:
loader = Loader()
df = loader.df
ser = df.treatment.value_counts()
ser


W    150
H     53
C     51
Name: treatment, dtype: int64